In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-06-27 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-27 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-27 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-27 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-27 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

In [7]:
x_data = obtain_path(
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    k=20
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

In [8]:
x_data.shape

(13551, 20, 384)

In [9]:
x_data[0]

array([[0.00154884, 0.01095446, 0.04541774, ..., 0.02605489, 0.05162514,
        0.0810306 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# Baseline: LSTM classification

In [10]:
num_epochs = 100
hidden_dim_sizes = [100, 200, 300]
num_layers = 1
bidirectional = True
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

## `history_length=20`

In [11]:
size = 20
bilstm_history_20_kfold, best_bilstm_history_20_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_20_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/lstm_history_20_focal_2_kfold_best_model.csv


In [12]:
bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_15788/3632055421.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  valid_accuracy  \
hidden_dim dropout_rate learning_rate                                       
100        0.1          0.0001         0.592803  0.468010        0.592803   
                        0.0005         0.594171  0.485014        0.594171   
                        0.0010         0.589621  0.473936        0.589621   
           0.2          0.0001         0.589859  0.466681        0.589859   
                        0.0005         0.594944  0.480964        0.594944   
                        0.0010         0.588045  0.474496        0.588045   
           0.5          0.0001         0.592506  0.474932        0.592506   
                        0.0005         0.591673  0.486742        0.591673   
                        0.0010         0.588729  0.485374        0.588729   
200        0.1          0.0001         0.600357  0.456879        0.600357   
                        0.0005         0.584952  0.472067        0.584952   
                        0.0010         0.587361  0.467919        0.587361   
           0.2          0.0001         0.595896  0.462770        0.595896   
                        0.0005         0.592892  0.476885        0.592892   
                        0.0010         0.585546  0.470537        0.585546   
           0.5          0.0001         0.603271  0.461372        0.603271   
                        0.0005         0.594706  0.485540        0.594706   
                        0.0010         0.601874  0.477086        0.601874   
300        0.1          0.0001         0.587450  0.461675        0.587450   
                        0.0005         0.591584  0.469673        0.591584   
                        0.0010         0.581561  0.454764        0.581561   
           0.2          0.0001         0.590007  0.464014        0.590007   
                        0.0005         0.589026  0.473806        0.589026   
                        0.0010         0.573710  0.456357        0.573710   
           0.5          0.0001         0.586082  0.471452        0.586082   
                        0.0005         0.598870  0.472808        0.598870   
                        0.0010         0.587212  0.454432        0.587212   

                                       valid_f1     k  num_layers  \
hidden_dim dropout_rate learning_rate                               
100        0.1          0.0001         0.468010  20.0         1.0   
                        0.0005         0.485014  20.0         1.0   
                        0.0010         0.473936  20.0         1.0   
           0.2          0.0001         0.466681  20.0         1.0   
                        0.0005         0.480964  20.0         1.0   
                        0.0010         0.474496  20.0         1.0   
           0.5          0.0001         0.474932  20.0         1.0   
                        0.0005         0.486742  20.0         1.0   
                        0.0010         0.485374  20.0         1.0   
200        0.1          0.0001         0.456879  20.0         1.0   
                        0.0005         0.472067  20.0         1.0   
                        0.0010         0.467919  20.0         1.0   
           0.2          0.0001         0.462770  20.0         1.0   
                        0.0005         0.476885  20.0         1.0   
                        0.0010         0.470537  20.0         1.0   
           0.5          0.0001         0.461372  20.0         1.0   
                        0.0005         0.485540  20.0         1.0   
                        0.0010         0.477086  20.0         1.0   
300        0.1          0.0001         0.461675  20.0         1.0   
                        0.0005         0.469673  20.0         1.0   
                        0.0010         0.454764  20.0         1.0   
           0.2          0.0001         0.464014  20.0         1.0   
                        0.0005         0.473806  20.0         1.0   
                        0.0010         0.456357  20.0         1.0   
           

In [13]:
best_bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,num_layers,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.597323,0.485214,"[0.7131677441306568, 0.3896791266953358, 0.352...",None,0.597323,0.485214,"[0.7131677441306568, 0.3896791266953358, 0.352...",1,True,100,0.5,0.0005,0,2,True
0,focal,0.581710,0.489577,"[0.6925558609152827, 0.4230999401555955, 0.353...",None,0.581710,0.489577,"[0.6925558609152827, 0.4230999401555955, 0.353...",1,True,100,0.5,0.0005,1,2,True
0,focal,0.593457,0.493252,"[0.70242656449553, 0.4186188013386066, 0.35870...",None,0.593457,0.493252,"[0.70242656449553, 0.4186188013386066, 0.35870...",1,True,100,0.5,0.0005,12,2,True
0,focal,0.576357,0.487224,"[0.6894566390788053, 0.4071013161922253, 0.365...",None,0.576357,0.487224,"[0.6894566390788053, 0.4071013161922253, 0.365...",1,True,100,0.5,0.0005,123,2,True
0,focal,0.609517,0.478445,"[0.7313140117672696, 0.33396728794218333, 0.37...",None,0.609517,0.478445,"[0.7313140117672696, 0.33396728794218333, 0.37...",1,True,100,0.5,0.0005,1234,2,True


In [14]:
best_bilstm_history_20_kfold["f1"].mean()

0.4867424679498775

In [15]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

array([0.70578416, 0.39449329, 0.35994995])

## `history_length=50`

In [ ]:
size = 50
bilstm_history_50_kfold, best_bilstm_history_50_kfold, _, __ = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
bilstm_history_50_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_50_kfold

In [ ]:
best_bilstm_history_50_kfold["f1"].mean()

In [ ]:
np.stack(best_bilstm_history_50_kfold["f1_scores"]).mean(axis=0)